### Лабораторная №1
Классификация тональности коротких текстов на основе TF-IDF и логистической
регрессии. При этом ребята должны "покрутить" морфологию (прежде всего, лемматизировать тексты, чтобы оперировать лексемами, а не словоформами), синтаксис (имеет смысл попробовать выделять именные группы вместо "слепых" биграмм или триграмм), коллокации и т.п. Для логрегрессии тоже надо попробовать применить AutoML - например, подобрать гиперпараметры с помощью банального BayesSearchCV https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html. Основной инструмент - scikit-learn и его друзья для фича-экстрактинга и классификации, udpipe для морфо-синтаксического анализа, gensim для коллокаций

In [ ]:
!pip install pymorphy2


In [ ]:
!pip install eli5

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dtype = {'id': np.uint16, 'date': str, 'name': str, 'text': str,  'type': np.int}
positive_df = pd.read_csv('/content/positive.csv', sep=";",  names=['id', 'date', 'name', 'text', 'type'], usecols=dtype.keys(), dtype=dtype)
negative_df = pd.read_csv('/content/negative.csv', sep=";",  names=['id', 'date', 'name', 'text', 'type'], usecols=dtype.keys(), dtype=dtype)

In [ ]:
positive_df.head()

,id,date,name,text,type
0,16384,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1
1,24577,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1
2,4097,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,0,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,2,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [ ]:
negative_df.head()

,id,date,name,text,type
0,8192,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1
1,24576,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1
2,0,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1
3,24576,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1
4,28672,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1


In [ ]:
positive_df.columns

Index(['id', 'date', 'name', 'text', 'type'], dtype='object')

In [ ]:
positive_df.drop(['id', 'date', 'name'], axis=1, inplace=True)
negative_df.drop(['id', 'date', 'name'], axis=1, inplace=True)

df = pd.concat([positive_df, negative_df], ignore_index=True)
df

,text,type
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,-1
226830,скучаю так :-( только @taaannyaaa вправляет мо...,-1
226831,"Вот и в школу, в говно это идти уже надо(",-1
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1


In [ ]:
df.rename(columns={'text': 'review', 'type': 'sentiment'}, inplace=True)
df

,review,sentiment
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,-1
226830,скучаю так :-( только @taaannyaaa вправляет мо...,-1
226831,"Вот и в школу, в говно это идти уже надо(",-1
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1


In [ ]:
df['sentiment'].value_counts()

 1    114911
-1    111923
Name: sentiment, dtype: int64

In [ ]:
df['review'].value_counts()

Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!                                                  156
RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT                                                                                                                 105
RT @Creative_Link: Дождались! Официальный трейлер "Иван Царевич и Серый Волк - 2". Смотреть всем: http://t.co/gmelSGMIuW Толи еще будет)                                 74
sm&amp;gt; напомни, как называется: фотка, вокруг черная рамка, а внизу - надпись?\nau&amp;gt; некролог?\nsm&amp;gt; не.. (\nau&amp;gt; демотиватор?\nsm&amp;gt; ДА!     70
RT @Podslyshano: люблю зиму за то, что люди носом разблокируют телефоны, и такие оглядываются типа без палева)) а я то все вижу. да да.                                  53
                                                                                                                                            

In [ ]:
df

,review,sentiment
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1
...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,-1
226830,скучаю так :-( только @taaannyaaa вправляет мо...,-1
226831,"Вот и в школу, в говно это идти уже надо(",-1
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1


In [ ]:
import pymorphy2
from tqdm import tqdm
tqdm.pandas()
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
import eli5

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   review     226834 non-null  object
 1   sentiment  226834 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.5+ MB


In [ ]:
df['sentiment'].value_counts()

 1    114911
-1    111923
Name: sentiment, dtype: int64

In [ ]:
df['review'].value_counts()

Офигенный день!\nдень позитива)\nбегал как идиот целый день!\nтанцы офигенны, хоть я и ракал но мне очень понравилось!                                                  156
RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT                                                                                                                 105
RT @Creative_Link: Дождались! Официальный трейлер "Иван Царевич и Серый Волк - 2". Смотреть всем: http://t.co/gmelSGMIuW Толи еще будет)                                 74
sm&amp;gt; напомни, как называется: фотка, вокруг черная рамка, а внизу - надпись?\nau&amp;gt; некролог?\nsm&amp;gt; не.. (\nau&amp;gt; демотиватор?\nsm&amp;gt; ДА!     70
RT @Podslyshano: люблю зиму за то, что люди носом разблокируют телефоны, и такие оглядываются типа без палева)) а я то все вижу. да да.                                  53
                                                                                                                                            

In [ ]:
df['review_processed'] = df['review'].apply(lambda x: (re.sub(r'[^\w\s]', '', x))) # знаки препинания



In [ ]:
df['review'].iloc[0]

'@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'

In [ ]:
 #оставляем только русские буквы
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^а-яА-Я ]',r'',x))

In [ ]:
df['review_processed'].iloc[0]

' хоть я и школота но поверь у нас то же самое  общество профилирующий предмет типа'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#токенизация

df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))

100%|██████████| 226834/226834 [00:34<00:00, 6620.08it/s]


In [ ]:
df['review_processed'].values

array([list(['хоть', 'я', 'и', 'школота', 'но', 'поверь', 'у', 'нас', 'то', 'же', 'самое', 'общество', 'профилирующий', 'предмет', 'типа']),
       list(['Да', 'всетаки', 'он', 'немного', 'похож', 'на', 'него', 'Но', 'мой', 'мальчик', 'все', 'равно', 'лучше']),
       list(['Ну', 'ты', 'идиотка', 'я', 'испугалась', 'за', 'тебя']),
       ...,
       list(['Вот', 'и', 'в', 'школу', 'в', 'говно', 'это', 'идти', 'уже', 'надо']),
       list(['Тауриэль', 'не', 'грусти', 'обнял']),
       list(['Такси', 'везет', 'меня', 'на', 'работу', 'Раздумываю', 'приплатить', 'чтобы', 'меня', 'втащили', 'на', 'пятый', 'этаж', 'Лифта', 'то', 'нет'])],
      dtype=object)

In [ ]:
# лемматизируем

morph = pymorphy2.MorphAnalyzer()
morph.parse("сделали")

[Parse(word='сделали', tag=OpencorporaTag('VERB,perf,tran plur,past,indc'), normal_form='сделать', score=1.0, methods_stack=((DictionaryAnalyzer(), 'сделали', 648, 4),))]

In [ ]:
morph.parse("сделали")[0].normal_form

'сделать'

In [ ]:
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda x: [morph.parse(w)[0].normal_form for w in x])

100%|██████████| 226834/226834 [09:09<00:00, 412.77it/s]


In [ ]:
df['review_lemmatized'].iloc[0]

['хоть',
 'я',
 'и',
 'школотый',
 'но',
 'поверь',
 'у',
 'мы',
 'то',
 'же',
 'самый',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [ ]:
# tf - idf
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(df['review_lemmatized'].progress_apply(lambda x: ' '.join(x)))

100%|██████████| 226834/226834 [00:00<00:00, 670338.49it/s]


In [ ]:
X.shape

(226834, 167721)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

In [ ]:
logit = LogisticRegression(random_state=15)

In [ ]:
logit.fit(X_train, y_train)
y_pred = logit.predict_proba(X_test)

In [ ]:
roc_auc_score(y_test, y_pred[:, 1]) # без n-грамм

0.8012891922463911

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
logit = LogisticRegression(random_state=15)
logit.fit(X_train, y_train)
y_pred = logit.predict_proba(X_test)
roc = roc_auc_score(y_test, y_pred[:, 1]) # с n-грамм

In [ ]:
roc

0.8201516262659565

In [ ]:
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names()), top=(20,20))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+5.960,ахи
+5.445,приятно
+5.416,ахах
+5.284,ахахи
+4.983,добрый
+4.627,спасибо
+4.466,обожать
+4.278,хах
+4.228,отличный
+4.162,рада


In [ ]:
from gensim.models import Phrases
document = ["Нью Йорк большой город", "Нью Йорк", "Поеду в Нью Йорк на следующей неделе"]
sent_stream = [doc.split(" ") for doc in document]
bigram = Phrases(sent_stream, min_count=2, threshold=1)
sent = ["Он","поехал","в","Нью","Йорк"]
print(bigram[sent])

['Он', 'поехал', 'в', 'Нью_Йорк']


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
df['review_lemmatized_collocate'] = df['review_lemmatized'].apply(lambda x: ' '.join(x))
sent_stream = [doc.split(" ") for doc in df['review_lemmatized_collocate']]
bigram = Phrases(sent_stream, min_count=5, threshold=2)
bigram[df["review_lemmatized"].iloc[0]]

['хоть',
 'я',
 'и',
 'школотый',
 'но',
 'поверь',
 'у_мы',
 'то',
 'же',
 'самый',
 'общество',
 'профилировать',
 'предмет',
 'тип']

In [ ]:
df['review_ngrams'] = df['review_lemmatized'].progress_apply(lambda x: bigram[x])

100%|██████████| 226834/226834 [00:11<00:00, 20618.62it/s]


In [ ]:
df['review_ngrams']

0         [хоть, я, и, школотый, но, поверь, у_мы, то, ж...
1         [да, всетаки, он, немного, похожий_на, он, но,...
2                  [ну, ты, идиотка, я, испугаться, за, ты]
3         [кто_то, в_угол, сидеть, и, погибать, от_голод...
4         [вот, что_значит, страшилка, но, блинпосмотрет...
                                ...                        
226829         [но, не, каждый, хотеть, что_то, исправлять]
226830    [скучать, так, только, вправлять, мозг, но, я,...
226831    [вот, и, в_школа, в, говно, это, идти, уже, надо]
226832                        [тауриэль, не_грусть, обнять]
226833    [такси, везти, я, на_работа, раздумывать, прип...
Name: review_ngrams, Length: 226834, dtype: object

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,1)) # только униграммы так как есть коллокации

X = vectorizer.fit_transform(df['review_ngrams'].apply(lambda x: ' '.join(x)))
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

logit = LogisticRegression(random_state=15)
logit.fit(X_train, y_train)

y_pred = logit.predict_proba(X_test)

roc = roc_auc_score(y_test, y_pred[:, 1]) # с n-грамм

In [ ]:
roc

0.8092450339523221

In [ ]:
eli5.show_weights(estimator=logit, feature_names=list(vectorizer.get_feature_names()), top=(20,20))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+4.392,ахахи
+4.146,ахи
+3.962,ахах
+3.692,хах
+3.549,приятно
+3.447,обожать
+3.307,ура
+3.281,смех
+3.247,чем_украсить
+3.192,ты_лка


In [ ]:
parameters={"C":[0.01, 0.1, 1, 10, 20], "penalty":["l1","l2"]}# l1 lasso l2 ridge
clf=GridSearchCV(logit, parameters)
clf.fit(X_train,y_train)

In [ ]:
print("tuned hpyerparameters :(best parameters) ",clf.best_params_)


tuned hpyerparameters :(best parameters)  {'C': 1, 'penalty': 'l2'}


In [ ]:
y_pred = clf.predict_proba(X_test)
roc = roc_auc_score(y_test, y_pred[:, 1])
roc

0.8092450339523221